In [1]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 1. 파일 불러오기

In [2]:
# 파일 불러오기
sheet_name = "input_test"
input_path = os.path.join(os.getcwd(), "input\\")
# input_test = pd.read_excel(input_path+"test_emoti_dup_korDup.xlsx", sheet_name=sheet_name, header=0)
input_test = pd.read_excel(input_path+"test.xlsx", sheet_name=sheet_name, header=0)
# null 제거
for i in range(len(input_test.columns)):
    input_test[input_test.columns[i]] = input_test[input_test.columns[i]].fillna("")
print(input_test.shape)
input_test.head(5)

(1364, 14)


,No,용어명,용어명 분리,용어설명,영문명,영문약어명,도메인명,데이터타입,데이터길이,소수점이하,생성일(제정일),범정부표준여부,정보시스템(DB),관련필드/용어
0,1,가입일자,가입_일자,"조직이나 단체 따위에 들어가거나, 서비스를 제공하는 상품 따위를 신청한 날짜를 연월...",,JOIN_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,
1,2,개설일자,개설_일자,설비나 제도 따위를 새로 마련하고 그에 관한 일을 처음 시작한 날짜를 연월일로 표현,,ESTBL_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,
2,3,개시일자,개시_일자,시장을 처음 열어 물건의 매매를 시작한 날짜를 연월일로 표현,,FTRSN_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,
3,4,개업일자,개업_일자,영업을 처음 시작한 날짜를 연월일로 표현,,OPBIZ_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,
4,5,개인정보 동의여부,개인_정보_동의_여부,특정 개인을 식별하거나 유추해 낼 수 있는 모든 정보를 공동 이용하도록 허락할지 참...,,INDV_INFO_AGRE_YN,여부C1,CHAR,1,,2020-08-13,범정부표준,,


# 2. 용어 분리

In [3]:
# 인덱스 컬럼 초기화
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']

In [4]:
# 용어명 분리

# 용어명 분리
input_split_kor = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist()).fillna("")

# 영문명 없을 경우 대비하여 cnt 집계
kor_firts_cnt = len(input_split_kor.columns)

# 컬럼명 입력
col_list_kor = []
for i in range(len(input_split_kor.columns)):
    col_list_kor.append("stwd_kor{}".format(i))
input_split_kor.columns = col_list_kor

# 한글 개수 세기
dupl_list_kor = []
dupl_cnt_kor = []
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = []
    for j in range(len(input_split_kor["stwd_kor{}".format(i)])):
        dupl_list_kor = input_split_kor["stwd_kor{}".format(i)].to_list().count(input_split_kor["stwd_kor{}".format(i)][j])
        globals()["dupl_list_kor{}".format(i)].append(dupl_list_kor)
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_kor{}".format(i)])], axis = 0)
    globals()["dupl_list_kor{}".format(i)].columns = ["dupl_list_kor{}".format(i)]
    
# 컬럼명,index, 단어위치 추가
for i in range(len(input_split_kor.columns)):
    input_split_kor["num_kor{}".format(i)] = "num_kor{}".format(i)
    globals()["stwd_kor{}".format(i)] = pd.concat(
            [
            df_index.fillna(""), 
            pd.DataFrame(input_split_kor["stwd_kor{}".format(i)]).fillna(""),
            pd.DataFrame(input_split_kor["num_kor{}".format(i)]).fillna(""),
            globals()["dupl_list_kor{}".format(i)].fillna("")
            ], 
            axis=1)
print(col_list_kor)
print(kor_firts_cnt)
# stwd_kor9
# dupl_list_kor0

['stwd_kor0', 'stwd_kor1', 'stwd_kor2', 'stwd_kor3', 'stwd_kor4', 'stwd_kor5', 'stwd_kor6', 'stwd_kor7']
8


In [5]:
len(input_test['영문약어명']) >= 0

True

In [6]:
# 영문명 분리

# 영문명이 하나라도 입력돼 있을 경우
if len(input_test['영문약어명']) > 0:
    print("영문명 컬럼 수 1개 이상:"+str((input_test['영문약어명']=="").sum()))

    # 영문명 분리
    input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist()).fillna("")
    
    # 영문명 컬럼 수
    eng_firts_cnt = len(input_split_eng.columns)
    
    # 컬럼명 입력
    col_list_eng = []
    for i in range(len(input_split_eng.columns)):
        col_list_eng.append("stwd_eng{}".format(i))
    input_split_eng.columns = col_list_eng
    
    # 영문 개수 세기
    dupl_list_eng = []
    dupl_cnt_eng = []
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = []
        for j in range(len(input_split_eng["stwd_eng{}".format(i)])):
            dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
            globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
        globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]
        
    # 컬럼명,index, 단어위치 추가
    for i in range(eng_firts_cnt):
        input_split_eng["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)] = pd.concat(
                [
                df_index.fillna(""), 
                pd.DataFrame(input_split_eng["stwd_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                globals()["dupl_list_eng{}".format(i)].fillna("")
                ], 
                axis=1)
        
# 영문명이 없을 경우
elif len(input_test['영문약어명']) == 0:
    print("영문명 컬럼 수 0개:"+str((input_test['영문약어명']=="").sum()))
    
    col_list_eng = []
    for i in range(kor_firts_cnt):
        globals()["stwd_eng{}".format(i)] = pd.DataFrame(columns = ["stwd_eng{}".format(i)])
        globals()["num_eng{}".format(i)] = pd.DataFrame(columns = ["num_eng{}".format(i)])
        globals()["dupl_list_eng{}".format(i)] = pd.DataFrame(columns = ["dupl_list_eng{}".format(i)])
        globals()["stwd_eng{}".format(i)] = pd.concat(
            [
                df_index.fillna(""), 
                globals()["stwd_eng{}".format(i)].fillna(""),
                globals()["num_eng{}".format(i)].fillna(""),
                globals()["dupl_list_eng{}".format(i)].fillna(""),
            ]
            , axis = 1
        )
        globals()["stwd_eng{}".format(i)]["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)]["dupl_list_eng{}".format(i)] = len(globals()["stwd_eng{}".format(i)])
        col_list_eng.append("stwd_eng{}".format(i))
        
else:
    print("한글컬럼, 영문커럼 예외 사항")

# 컬럼수 일치 점검
col_cnt_kor = len(col_list_kor)
col_cnt_eng = len(col_list_eng)
if len(col_list_kor) == len(col_list_eng):
    print("한글컬럼수와 영문컬럼수 일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
else:
    print("한글컬럼수와 영문컬럼수 불일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))                                                       
print(col_list_eng)
stwd_eng0

영문명 컬럼 수 1개 이상:0
한글컬럼수와 영문컬럼수 불일치
kor:8 vs eng:6
['stwd_eng0', 'stwd_eng1', 'stwd_eng2', 'stwd_eng3', 'stwd_eng4', 'stwd_eng5']


,index,stwd_eng0,num_eng0,dupl_list_eng0
0,1,JOIN,num_eng0,1
1,2,ESTBL,num_eng0,1
2,3,FTRSN,num_eng0,1
3,4,OPBIZ,num_eng0,1
4,5,INDV,num_eng0,1
...,...,...,...,...
1359,1360,FAMT,num_eng0,3
1360,1361,DCNT,num_eng0,13
1361,1362,PRC,num_eng0,20
1362,1363,PRC,num_eng0,20


In [7]:
# 한글/영문 컬럼 수 일치시키기

# 한글컬럼과 영문컬럼 수 일치할 경우
if col_cnt_kor == col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = None
    col_cnt_diff = None
    print('한글컬럼과 영문컬럼 수 일치')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
    
# 한글컬럼이 더 많을 경우
elif col_cnt_kor > col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = list(set(range(col_cnt_kor)) - set(range(col_cnt_eng)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        input_split_eng['dupl_list_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["dupl_list_eng{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('한글컬럼이 더 많은 싱태')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  

# 영문컬럼이 더 많을 경우
elif col_cnt_kor < col_cnt_eng:
    col_cnt = col_cnt_eng
    col_cnt_diff_list = list(set(range(col_cnt_eng)) - set(range(col_cnt_kor)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        input_split_eng['dupl_list_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)].fillna("")).
                pd.DataFrame(input_split_eng["dupl_list_kor{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('영문컬럼이 더 많은 싱태')       
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
stwd_eng3

한글컬럼이 더 많은 싱태
kor:8 vs eng:6
컬럼수 일치작업 후:8


,index,stwd_eng3,num_eng3,dupl_list_eng3
0,1,,num_eng3,978
1,2,,num_eng3,978
2,3,,num_eng3,978
3,4,,num_eng3,978
4,5,YN,num_eng3,3
...,...,...,...,...
1359,1360,TXT,num_eng3,41
1360,1361,,num_eng3,978
1361,1362,STT,num_eng3,2
1362,1363,STT,num_eng3,2


In [8]:
# n x m 정사각형 DafaFrame 점검
for i in range(col_cnt_kor):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = int(np.mean(cnt))
avg_avg_kor = int(np.mean(avg))
for i in range(col_cnt_eng):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = int(np.mean(cnt))
avg_avg_eng = int(np.mean(avg))
if avg_avg_kor == avg_avg_eng:
    print("한글 영문 row수 일치: 평균=" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글:" + str(avg_avg_kor)+", 영문:" + str(avg_avg_eng))
else:
    print("한글 영문 row수 불일치: 평균" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글:" + str(avg_avg_kor)+", =영문:" + str(avg_avg_eng))

한글 영문 row수 일치: 평균=1364.0
한글:1364, 영문:1364


In [9]:
# 중복검사를 위한 left join
for i in range(col_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )
print(col_cnt)
# stwd_total9

8


In [10]:
# 일렬 세우기

# unique_stwd 초기화 
stwd_total = pd.DataFrame()

## 컬럼명 바꾸고  concat
for i in range(col_cnt):
    globals()["stwd_total{}".format(i)].columns = [
        'index', 'stwd_kor', 'num_kor', 'dupl_list_kor', 'stwd_eng','num_eng', 'dupl_list_eng'
    ]
    stwd_total = pd.concat([stwd_total, globals()["stwd_total{}".format(i)]], axis = 0, ignore_index = True)
print(stwd_total.shape)
stwd_total

(10912, 7)


,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng
0,1,가입,num_kor0,1,JOIN,num_eng0,1
1,2,개설,num_kor0,1,ESTBL,num_eng0,1
2,3,개시,num_kor0,1,FTRSN,num_eng0,1
3,4,개업,num_kor0,1,OPBIZ,num_eng0,1
4,5,개인,num_kor0,1,INDV,num_eng0,1
...,...,...,...,...,...,...,...
10907,1360,,num_kor7,1364,,,
10908,1361,,num_kor7,1364,,,
10909,1362,,num_kor7,1364,,,
10910,1363,,num_kor7,1364,,,


# 3. 이상값 수정

In [11]:
# 특수기호 제거

# (테스트) 특수기호 제거 
stwd_total.loc[0, 'stwd_kor'] = stwd_total.loc[0, 'stwd_kor'] + '!!@(#$!@$!@$'
print('[test] 적용 전: ' + stwd_total.loc[0, 'stwd_kor'])

# 한글 특수기호 제거
stwd_total_cnt = len(stwd_total['stwd_kor'])
for i in range(stwd_total_cnt):
    stwd_total.loc[i, 'stwd_kor'] =  stwd_total['stwd_kor'][i].translate(str.maketrans('', '', string.punctuation))
#     stwd_total['stwd_kor'][i] = stwd_total['stwd_kor'][i].translate(str.maketrans('', '', string.punctuation)) # 안됨!!!
    cnt += i
print('[test] 적용 후: ' + str(stwd_total.loc[0, 'stwd_kor']))
print(cnt)
stwd_total

# (테스트) 특수기호 제거
stwd_total.loc[6, 'stwd_eng'] = str(stwd_total.loc[6, 'stwd_eng']) + '!@#$_'
print('[test] 적용 전: ' + stwd_total.loc[6, 'stwd_eng'])

# 영문 특수기호 제거
stwd_total_cnt = len(stwd_total['stwd_eng'])
for i in range(stwd_total_cnt):
    stwd_total.loc[i, 'stwd_eng'] =  stwd_total['stwd_eng'][i].translate(str.maketrans('', '', string.punctuation))
    cnt += i
print('[test] 적용 후: ' + str(stwd_total.loc[6, 'stwd_eng']))
print(cnt)
stwd_total

[test] 적용 전: 가입!!@(#$!@$!@$
[test] 적용 후: 가입
59531780
[test] 적용 전: RNWL!@#$_
[test] 적용 후: RNWL
119062196


,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng
0,1,가입,num_kor0,1,JOIN,num_eng0,1
1,2,개설,num_kor0,1,ESTBL,num_eng0,1
2,3,개시,num_kor0,1,FTRSN,num_eng0,1
3,4,개업,num_kor0,1,OPBIZ,num_eng0,1
4,5,개인,num_kor0,1,INDV,num_eng0,1
...,...,...,...,...,...,...,...
10907,1360,,num_kor7,1364,,,
10908,1361,,num_kor7,1364,,,
10909,1362,,num_kor7,1364,,,
10910,1363,,num_kor7,1364,,,


In [12]:
# 한글 소문자 탐색 
test = 'test'
stwd_total.loc[2, 'stwd_kor'] = stwd_total.loc[2, 'stwd_kor'] + test
print('[test] 적용 전(가짜): ' + stwd_total.loc[2, 'stwd_kor'])
df_lower_kor = pd.DataFrame(columns = stwd_total.columns)
for i in range(stwd_total_cnt):
    if stwd_total['stwd_kor'][i].islower():
        tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
        df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
stwd_total['stwd_kor_old'] = stwd_total['stwd_kor']
stwd_total['stwd_kor'] = stwd_total['stwd_kor'].str.upper()

# 적용 후 결과값
print('[test] 적용 후(대문자): ' + stwd_total.loc[2, 'stwd_kor'])
stwd_total.loc[2, 'stwd_kor'] = stwd_total.loc[2, 'stwd_kor'].strip(test.upper())
print('[test] 복구: 개시')
print('변경된 값: ' + str(len(df_lower_kor)) + '개')
stwd_total.head()

[test] 적용 전(가짜): 개시test
[test] 적용 후(대문자): 개시TEST
[test] 복구: 개시
변경된 값: 1개


,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng,stwd_kor_old
0,1,가입,num_kor0,1,JOIN,num_eng0,1,가입
1,2,개설,num_kor0,1,ESTBL,num_eng0,1,개설
2,3,개시,num_kor0,1,FTRSN,num_eng0,1,개시test
3,4,개업,num_kor0,1,OPBIZ,num_eng0,1,개업
4,5,개인,num_kor0,1,INDV,num_eng0,1,개인


In [13]:
# 영문 소문자 탐색 
print('[test] 적용 전(소문자): ' + stwd_total.loc[2, 'stwd_eng'])
df_lower_eng = pd.DataFrame(columns = stwd_total.columns)
for i in range(stwd_total_cnt):
    if stwd_total['stwd_eng'][i].islower():
        tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
        df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
stwd_total['stwd_eng_old'] = stwd_total['stwd_eng']
stwd_total['stwd_eng'] = stwd_total['stwd_eng'].str.upper()

# 적용 후 결과값
print('[test] 적용 후(대문자): ' + stwd_total.loc[2, 'stwd_eng'])
print('변경된 값: ' + str(len(df_lower_eng)) + '개')
stwd_total.head()

[test] 적용 전(소문자): FTRSN
[test] 적용 후(대문자): FTRSN
변경된 값: 0개


,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng,stwd_kor_old,stwd_eng_old
0,1,가입,num_kor0,1,JOIN,num_eng0,1,가입,JOIN
1,2,개설,num_kor0,1,ESTBL,num_eng0,1,개설,ESTBL
2,3,개시,num_kor0,1,FTRSN,num_eng0,1,개시test,FTRSN
3,4,개업,num_kor0,1,OPBIZ,num_eng0,1,개업,OPBIZ
4,5,개인,num_kor0,1,INDV,num_eng0,1,개인,INDV


In [14]:
# 띄어쓰기 제거(다시)
stwd_total['stwd_kor'] = stwd_total['stwd_kor'].str.replace(" ", "")
stwd_total['stwd_eng'] = stwd_total['stwd_eng'].str.replace(" ", "")
print(stwd_total.shape)
stwd_total.head(500)

(10912, 9)


,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng,stwd_kor_old,stwd_eng_old
0,1,가입,num_kor0,1,JOIN,num_eng0,1,가입,JOIN
1,2,개설,num_kor0,1,ESTBL,num_eng0,1,개설,ESTBL
2,3,개시,num_kor0,1,FTRSN,num_eng0,1,개시test,FTRSN
3,4,개업,num_kor0,1,OPBIZ,num_eng0,1,개업,OPBIZ
4,5,개인,num_kor0,1,INDV,num_eng0,1,개인,INDV
5,6,개정,num_kor0,1,RVSN,num_eng0,1,개정,RVSN
6,7,갱신,num_kor0,1,RNWL,num_eng0,1,갱신,RNWL
7,8,거래,num_kor0,1,DLNG,num_eng0,1,거래,DLNG
8,9,검사,num_kor0,1,INSP,num_eng0,1,검사,INSP
9,10,검증,num_kor0,1,VRFC,num_eng0,1,검증,VRFC


# 한글명에 한글과 영문대문자 이외에는 모두 삭제 로직

In [15]:
# 영문명에 한글명 포함되었는가 탐색\
        
# 한글 자동 처리
print('[자동 처리 시작합니다]')
exKor_list = []
stwd_total['stwd_eng_exKor'] = ''
for i in range(len(stwd_total)):
    cor = re.sub('[^a-zA-Z]', '', stwd_total['stwd_eng'][i])
    chk = re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', stwd_total['stwd_eng'][i])
    if bool(chk):
        print("-----------------------------------------------------------------|")
        print('index: ' + str(i))
        print(stwd_total['stwd_eng'][i] + " -> " + str(''.join(chk)))
        exKor_list.append(i)
    stwd_total.loc[i,'stwd_eng_exKor'] = cor
stwd_total_manu1 = stwd_total.iloc[exKor_list]
print('[자동 처리 완료하였습니다]')


    ################# 파일 생성 처리 로직########################

[자동 처리 시작합니다]
[자동 처리 완료하였습니다]


In [16]:
# 한글처리 묻기
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print('실패: y 혹은 n 입력')
            
if yes_or_no('영문명의 한글 제거 처리를 적용 하시겠습니까?'):
    stwd_total['stwd_eng'] = stwd_total['stwd_eng_exKor']
    print('[적용되었습니다]')
else:
    file_name = 'stwd_total_manu1'
    print('[엑셀 파일을 생성하였습니다]: ' + file_name +'_'+ now)
    print('[검토한 후 다시 업로드 해주세요]')
stwd_total.head()

영문명의 한글 제거 처리를 적용 하시겠습니까? (y/n): y
[적용되었습니다]


,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng,stwd_kor_old,stwd_eng_old,stwd_eng_exKor
0,1,가입,num_kor0,1,JOIN,num_eng0,1,가입,JOIN,JOIN
1,2,개설,num_kor0,1,ESTBL,num_eng0,1,개설,ESTBL,ESTBL
2,3,개시,num_kor0,1,FTRSN,num_eng0,1,개시test,FTRSN,FTRSN
3,4,개업,num_kor0,1,OPBIZ,num_eng0,1,개업,OPBIZ,OPBIZ
4,5,개인,num_kor0,1,INDV,num_eng0,1,개인,INDV,INDV


In [17]:
# 각각 중복제거 대신에... 한글+영문 아이디 만들어 이걸로 중복제거! + 한글명 정렬!!!

In [20]:
# # 우선 그대로 원복하고, index list 만들어서 조회만..?
# stwd_total['id'] = stwd_total['stwd_kor'] + stwd_total['stwd_eng']

# print(stwd_unique.shape)
# stwd_unique
# stwd_kor_unique = stwd_unique[['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']]
# stwd_eng_unique = stwd_unique[['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']]
# stwd_kor_unique

In [21]:
# stwd_kor_unique = stwd_unique[['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']]
# stwd_eng_unique = stwd_unique[['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']]
# stwd_kor_unique

In [22]:
# (일렬)전체 그대로
stwd_total

,index,stwd_kor,num_kor,dupl_list_kor,stwd_eng,num_eng,dupl_list_eng,stwd_kor_old,stwd_eng_old,stwd_eng_exKor,id
0,1,가입,num_kor0,1,JOIN,num_eng0,1,가입,JOIN,JOIN,가입JOIN
1,2,개설,num_kor0,1,ESTBL,num_eng0,1,개설,ESTBL,ESTBL,개설ESTBL
2,3,개시,num_kor0,1,FTRSN,num_eng0,1,개시test,FTRSN,FTRSN,개시FTRSN
3,4,개업,num_kor0,1,OPBIZ,num_eng0,1,개업,OPBIZ,OPBIZ,개업OPBIZ
4,5,개인,num_kor0,1,INDV,num_eng0,1,개인,INDV,INDV,개인INDV
...,...,...,...,...,...,...,...,...,...,...,...
10907,1360,,num_kor7,1364,,,,,,,
10908,1361,,num_kor7,1364,,,,,,,
10909,1362,,num_kor7,1364,,,,,,,
10910,1363,,num_kor7,1364,,,,,,,


# 중복제거 로직

In [23]:
# 중복제거 후 다시 합쳐서 검사(2안 + 1안)
# 한글 중복제거
col_stwd_kor = ['stwd_kor', 'num_kor']
unique_kor = stwd_total['stwd_kor'].drop_duplicates()
stwd_kor_unique = stwd_total.loc[unique_kor.index][col_stwd_kor]
print(stwd_kor_unique.shape)
# stwd_kor_unique

# 영문 중복제거
col_stwd_eng = ['stwd_eng', 'num_eng']
unique_eng = stwd_total['stwd_eng'].drop_duplicates()
stwd_eng_unique = stwd_total.loc[unique_eng.index][col_stwd_eng]
print(stwd_eng_unique.shape)
# stwd_eng_unique.tail(100)

# 한글+영문 일렬 세우기
df_step_1 = pd.DataFrame(stwd_total['index'])
df_step_1 = df_step_1.merge(
    stwd_kor_unique,
    left_index = True,
    right_index = True,
    how = 'left'
).merge(
    stwd_eng_unique,
    left_index = True,
    right_index = True,
    how = 'left'
)
df_step_1 = df_step_1[['index', 'stwd_kor', 'stwd_eng', 'num_kor', 'num_eng']]

# unique 뽑기 & null 값 제거
df_step_uniq = df_step_1.dropna(axis = 0)
df_step_uniq = df_step_uniq.sort_values(['stwd_kor', 'stwd_eng'])
print(df_step_uniq.shape)

df_step_uniq = df_step_uniq.merge(input_test, left_on='index', right_on='No', how='left')
print(df_step_uniq.shape)

df_step_uniq.tail(500)

(679, 2)
(702, 2)
(521, 5)
(521, 19)


,index,stwd_kor,stwd_eng,num_kor,num_eng,No,용어명,용어명 분리,용어설명,영문명,영문약어명,도메인명,데이터타입,데이터길이,소수점이하,생성일(제정일),범정부표준여부,정보시스템(DB),관련필드/용어
21,829,개황,OTLK,num_kor2,num_eng2,829,이례사항개황내용,이례_사항_개황_내용,한국철도공사 열차운행 중 발생한 이례사항 개황내용,,EXCP_ATCL_OTLK_CONT,내역V2000,VARCHAR,2000,,2021-07-01,코레일_기관표준,XROIS,
22,369,객차,PSC,num_kor0,num_eng0,369,객차 연결량수,객차_연결량_수,한국철도공사 연결된 객차의 차량수,,PSC_CPL_CNT,수D5,DEC,5,,2021-07-01,코레일_기관표준,XROIS,
23,7,갱신,RNWL,num_kor0,num_eng0,7,갱신일자,갱신_일자,이미 있던 것을 고쳐 새롭게 한 날짜를 연월일로 표현,,RNWL_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,
24,8,거래,DLNG,num_kor0,num_eng0,8,거래일자,거래_일자,주고받거나 사고판 날짜를 연월일로 표현,,DLNG_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,
25,616,거리,DST,num_kor1,num_eng1,616,세그먼트거리,세그먼트_거리,한국철도공사 세그먼트 거리 (단위 Km),,SGMT_DST,"거리D8,3",DEC,8,3.0,2021-07-01,코레일_기관표준,XROIS,
26,640,건수,IMP,num_kor1,num_eng1,640,수입건수,수입_건수,관세청에서 발표하는 수출입총괄표의 수입건수,,KCS_IMP_CNT,수D15,DEC,15,0.0,2021-07-01,코레일_기관표준,관세청 API,
27,9,검사,INSP,num_kor0,num_eng0,9,검사일자,검사_일자,사실이나 일의 상태 또는 물질의 구성 성분 따위를 조사하여 옳고 그름과 낫고 못함을...,,INSP_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,
28,377,검사자,OVH,num_kor0,num_eng0,377,검사자사원번호,검사자_사원_번호,한국철도공사 열차 검사자 사원번호,,OVH_PS_EPNO,사원번호C6,CHAR,6,,2021-07-01,코레일_기관표준,XROIS,
29,378,검수도중,ISP,num_kor0,num_eng0,378,검수도중기관정지시간수,검수도중_기관정지_시간_수,한국철도공사 검수도중기관정지시간수,,ISP_ITMT_ISTT_HALT_TNUM,시간수I11,INT,11,,2021-07-01,코레일_기관표준,XROIS,
30,10,검증,VRFC,num_kor0,num_eng0,10,검증일자,검증_일자,"업무, 프로세스, 프로그램, 데이터 등의 오류 및 정합성을 확인한 날짜를 연월일로 표현",,VRFC_YMD,연월일C8,CHAR,8,,2020-08-13,범정부표준,,


In [27]:
df_step_uniq.to_excel(input_path + "220928.xlsx")

# 여기까지

In [ ]:
## 파일 생성 -> 중복 없어질때까지 n회 반복

## step1.1. 끝
일렬 나열 및 단순 중복 제거

## step1.2. 시작
스텝 1.2. 는 중복제거 한 파일을 ID를 기준으로 다시 중복제거 한 후, 재점검하여, 아닌 것 같은 경우는 초기 input 파일(원본)처럼 만듦

In [ ]:
stwd_total

In [ ]:
pd.to_numeric(df_step_raw1['id_uniq_chk{}'.format(i)])

In [ ]:
# stwd_kor_unique = stwd_total.iloc[stwd_kor_unique.index]
# stwd_eng_unique = stwd_total.iloc[stwd_eng_unique.index]
stwd_eng_unique

In [ ]:
stwd_total['stwd_eng'][7289]

In [ ]:
stwd_total['stwd_eng'].drop_duplicates()

In [ ]:
# id 유니크 체크
# id 컬럼 만들기
stwd_total['id'] = stwd_total['stwd_kor'] + stwd_total['stwd_eng']

# uniq 항목 체크하기
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
stwd_total['uniq_chk'] = ''
stwd_total.loc[stwd_unique_list.index, 'uniq_chk'] = 1
stwd_total

In [ ]:
# 컬럼수만큼 테이블 만들고, 유니크 체그(한글명 id로만)
df_step_raw1 = df_index
for i in range(col_cnt):
    globals()['stwd_kor_uinq{}'.format(i)] = stwd_total[stwd_total['num_kor'] == 'num_kor{}'.format(i)]
    kor_list = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor', 'uniq_chk', 'id']
    globals()['stwd_kor_uinq{}'.format(i)] = globals()['stwd_kor_uinq{}'.format(i)][kor_list]
    globals()['stwd_kor_uinq{}'.format(i)].columns = [
        'index', 
        'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i),
        'id_uniq_chk{}'.format(i),
        'id{}'.format(i)
    ]
    globals()['stwd_eng_uinq{}'.format(i)] = stwd_total[stwd_total['num_eng'] == 'num_eng{}'.format(i)]
    eng_list = ['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']
    globals()['stwd_eng_uinq{}'.format(i)] = globals()['stwd_eng_uinq{}'.format(i)][eng_list]
    globals()['stwd_eng_uinq{}'.format(i)].columns = [
        'index', 
        'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i)
    ]
    # 한글,영문 통합하여 원본데이터 형태로
    df_step_raw1 = df_step_raw1.merge(
        globals()['stwd_kor_uinq{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uinq{}'.format(i)], 
        on ='index',
        how = 'left'
    )
    
# 각 row의 unique(id) 반복 횟수 적기 
id_uniq_chk_sum = 0
for i in range(col_cnt):
    globals()['id_uniq_chk{}'.format(i)] = pd.to_numeric(df_step_raw1['id_uniq_chk{}'.format(i)])
    id_uniq_chk_sum = id_uniq_chk_sum + globals()['id_uniq_chk{}'.format(i)].fillna(0)
id_uniq_chk_sum = pd.DataFrame(id_uniq_chk_sum, columns = ['id_uniq_chk_sum'])

# 계산값 df에 추가하기
df_step_raw1 = df_step_raw1.merge(
    id_uniq_chk_sum,
    left_index = True,
    right_index = True,
    how = 'left'
)

df_step_raw1

In [ ]:
df_step_raw1[tmp_col_list]

In [ ]:
# null 제거
df_step_1 = df_step_raw1[df_step_raw1['stwd_kor0'].notna()]

# 빈칸 제거
df_step_1 = df_step_1.loc[df_step_1['stwd_kor0'] != ""]

# 필요 컬럼 추리기
tmp_col_list = ['index', 'id_uniq_chk_sum']
for i in range(col_cnt):
    tmp_col_list.append('stwd_kor{}'.format(i))
    tmp_col_list.append('stwd_eng{}'.format(i))
    tmp_col_list.append('id_uniq_chk{}'.format(i))
    tmp_col_list.append('id{}'.format(i))
df_step_1 = df_step_1[tmp_col_list]

# row의 id 반복 횟수 적기(1 이상) 
df_step_1 = df_step_1.loc[df_step_1['id_uniq_chk_sum'] >= 1]

# 순서 정리
df_step_1 = df_step_1.sort_values(['stwd_kor0', 'stwd_eng0'])

print('카운트_전 :'+ str(df_step_raw1.shape))
print('카운트_후 :'+ str(df_step_1.shape))
# df_step_1.to_excel(input_path + "1step안_완.xlsx")
df_step_1.to_excel(input_path + "1step안_완Real.xlsx")
df_step_1.head(500)

In [ ]:

# 계산값 df에 추가하기
df_step_1 = df_step_1.merge(
    id_uniq_chk_sum,
    left_index = True,
    right_index = True,
    how = 'left'
)
df_step_1

In [ ]:
# 그냥 다시 합치기(한/영)
df_exam_raw1 = df_index

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total[stwd_total['num_kor'] == 'num_kor{}'.format(i)]
    kor_list = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']
    globals()['stwd_kor_uni{}'.format(i)] = globals()['stwd_kor_uni{}'.format(i)][kor_list]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_total[stwd_total['num_eng'] == 'num_eng{}'.format(i)]
    eng_list = ['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']
    globals()['stwd_eng_uni{}'.format(i)] = globals()['stwd_eng_uni{}'.format(i)][eng_list]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )
    

df_exam_raw1

In [ ]:

#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

df_exam_1 = df_exam_raw1.sort_values(['index', 'stwd_kor0'])
print(df_exam_1.shape)
df_exam_raw1.tail(50)
# df_exam_raw1.head(500)

In [ ]:
stwd_unique_list.index

In [ ]:
# (일렬)id 유니크 만 탐색
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
a = stwd_total.iloc[stwd_unique_list.index]
a


In [ ]:
stwd_unique

In [ ]:
# 우선 그대로 원복하고, index list 만들어서 조회만..?
stwd_total['id'] = stwd_total['stwd_kor'] + stwd_total['stwd_eng']
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
print(stwd_unique.shape)
stwd_unique
stwd_kor_unique = stwd_unique[['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']]
stwd_eng_unique = stwd_unique[['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']]
stwd_kor_unique

In [ ]:
# 다시 합치기(한/영)
df_exam_raw1 = a

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_eng_unique[stwd_eng_unique['num_eng'] == 'num_eng{}'.format(i)]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

df_exam_1 = df_exam_raw1.sort_values(['index', 'stwd_kor0'])
print(df_exam_1.shape)
df_exam_raw1.tail(50)
# df_exam_raw1.head(500)

In [ ]:
a.to_excel(input_path + "3안.xlsx")

In [ ]:
# [1안] 중복제거(새로운 id 셍상)
stwd_total['id'] = stwd_total['stwd_kor'] + stwd_total['stwd_eng']
stwd_unique_list = stwd_total['id'].drop_duplicates()
stwd_unique = stwd_total.loc[stwd_unique_list.index]
print(stwd_unique.shape)
stwd_unique
stwd_kor_unique = stwd_unique[['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']]
stwd_eng_unique = stwd_unique[['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']]
stwd_kor_unique

In [ ]:
# 다시 합치기(한/영)
df_exam_raw1 = df_index

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_eng_unique[stwd_eng_unique['num_eng'] == 'num_eng{}'.format(i)]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

df_exam_1 = df_exam_raw1.sort_values(['index', 'stwd_kor0'])
print(df_exam_1.shape)
df_exam_raw1.tail(50)
# df_exam_raw1.head(500)

In [ ]:
avg_avg_kor


# sdfsㄹ


In [ ]:
total_cnt = avg_avg_kor
null_list = []
for i in range(total_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if df_exam_1.iloc[i].notna().sum() > 1:
        null_list.append(i)
        
#         print(i)
df_exam_1 = df_exam_1.loc[null_list]
df_exam_1.to_excel(input_path + "1안.xlsx")
df_exam_1


In [ ]:
# df_id = pd.DataFrame(stwd_total['id'])
# df_exam_1 = df_exam_1.merge(df_id, left_index=True, right_index=True, how='left').tail()

In [ ]:
# # null 제거
# df_exam_1 = df_exam_raw1[df_exam_raw1['stwd_kor0'].notna()]

# # 빈칸 제거
# df_exam_1 = df_exam_1.loc[df_exam_1['stwd_kor0'] != ""]

# 필요 컬럼 추리기
tmp_col_list = ['index']
for i in range(col_cnt):
    tmp_col_list.append('stwd_kor{}'.format(i))
    tmp_col_list.append('stwd_eng{}'.format(i))
tmp_col_list

df_exam_1 = df_exam_1[tmp_col_list]
# df_exam_1 = pd.concat([df_exam_1, df_id] ,axis=1)
# id_unique_list = df_id['id'].drop_duplicates()
# df_exam_1 = df_exam_1.loc[id_unique_list.index]
print('2안과 카운트 비교:'+ str(df_exam_1.shape))
df_exam_1.to_excel(input_path + "1안.xlsx")
df_exam_1.head()



In [ ]:
# 2안: 각각 중복제거 후 다시 합쳐서 검사
# 한글 중복제거
col_stwd_kor = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']
unique_kor = stwd_total['stwd_kor'].drop_duplicates()
stwd_kor_unique = stwd_total.loc[unique_kor.index][col_stwd_kor]
print(stwd_kor_unique.shape)
stwd_kor_unique

In [ ]:
# 2안: 각각 중복제거 후 다시 합쳐서 검사
# 한글 중복제거
col_stwd_kor = ['index', 'stwd_kor', 'num_kor', 'dupl_list_kor']
unique_kor = stwd_total['stwd_kor'].drop_duplicates()
stwd_kor_unique = stwd_total.loc[unique_kor.index][col_stwd_kor]
print(stwd_kor_unique.shape)
stwd_kor_unique

# 영문 중복제거
col_stwd_eng = ['index', 'stwd_eng', 'num_eng', 'dupl_list_eng']
unique_eng = stwd_total['stwd_eng'].drop_duplicates()
stwd_eng_unique = stwd_total.loc[unique_eng.index][col_stwd_eng]
print(stwd_eng_unique.shape)
stwd_eng_unique

# 다시 합치기(한/영)
df_exam_raw1 = df_index

for i in range(col_cnt):
    globals()['stwd_kor_uni{}'.format(i)] = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = [
        'index', 'stwd_kor{}'.format(i), 
        'num_kor{}'.format(i), 
        'dupl_list_kor{}'.format(i) 
    ]
    globals()['stwd_eng_uni{}'.format(i)] = stwd_eng_unique[stwd_eng_unique['num_eng'] == 'num_eng{}'.format(i)]
    globals()['stwd_eng_uni{}'.format(i)].columns = [
        'index', 'stwd_eng{}'.format(i), 
        'num_eng{}'.format(i), 
        'dupl_list_eng{}'.format(i) 
    ]
#     log = stwd_kor_unique[stwd_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     print('===========================================================================')
#     print(log)
    df_exam_raw1 = df_exam_raw1.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        on = 'index',
        how = 'left'
    ).merge(
        globals()['stwd_eng_uni{}'.format(i)], 
        on ='index',
        how = 'left'
    )

print(df_exam_raw1.shape)
df_exam_raw1.head()

# null 제거
df_exam_1 = df_exam_1[df_exam_1.notna()]

# 빈칸 제거
df_exam_1 = df_exam_1.loc[df_exam_1['stwd_kor0'] != ""]

# 필요 컬럼 추리기
tmp_col_list = ['index']
for i in range(col_cnt):
    tmp_col_list.append('stwd_kor{}'.format(i))
    tmp_col_list.append('stwd_eng{}'.format(i))
tmp_col_list

df_exam_1 = df_exam_1[tmp_col_list]
print('1안과 카운트 비교:'+ str(df_exam_1.shape))
df_exam_1.to_excel(input_path + "2안.xlsx")
df_exam_1.head()

In [ ]:
## df_exam_1 와 원본 붙이기!!

In [ ]:
unique_kor

In [ ]:
df_exam_1.to_excel(input_path + "df_exam_1.xlsx")
df_exam_1.to_excel(input_path + "df_exam_1.xlsx")
stwd_total.to_excel(input_path + "stwd_total.xlsx")

In [ ]:
STOP!!!!!!!!!!

In [ ]:
stwd_total

In [ ]:
# 1차 검토
# 유니크(한 * 영) 데이터 뽑아 엑셀 생성
# 수정후 total 데이터 뽑아 엑셀 생성


In [ ]:
stwd_total_kor_unique

In [ ]:
stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]

In [ ]:
tt = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(0)]
tt['stwd_kor']
tt

In [ ]:
for i in range(kor_firts_cnt):
#     a = pd.DataFrame()
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    print(i)

In [ ]:
ba = df_index.merge(globals()['stwd_kor_uni{}'.format(0)], left_on = 'index', right_on = 'index', how = 'left')
ba

In [ ]:
# 다시 합치기
df_index_tmp = df_index

for i in range(col_cnt):
    a = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = ['index', 'stwd_kor{}'.format(i), 'num_kor{}'.format(i), 'dupl_list_kor{}'.format(i)]
#     tmp['num_kor{}'.format(i)] = a['stwd_kor']
    aa = a.merge(globals()['stwd_kor_uni{}'.format(i)], left_on = 'index', right_on = 'index', how = 'left')
    print('===========================================================================')
    print(a)
    df_index_tmp = df_index_tmp.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        left_on ='index',
        right_on = 'index',
        how = 'left'
    )


df_index_tmp 

In [ ]:
df_index_tmp

In [ ]:
df_index_tmp[df_index_tmp['stwd_kor{}'.format(0)].isna()]

In [ ]:
# null제거
for i in range(col_cnt):
    df_index_tmp['stwd_kor{}'.format(i)] = df_index_tmp['stwd_kor{}'.format(i)].fillna("")
    df_index_tmp['num_kor{}'.format(i)] = df_index_tmp['num_kor{}'.format(i)].fillna("")
    df_index_tmp['dupl_list_kor{}'.format(i)] = df_index_tmp['dupl_list_kor{}'.format(i)].fillna("")

In [ ]:
    
# null값 수정
for i in range(col_cnt):
    df_index_tmp['stwd_kor{}'.format(i)] = df_index_tmp['stwd_kor{}'.format(i)].fillna("")
    df_index_tmp['num_kor{}'.format(i)] = df_index_tmp['num_kor{}'.format(i)].fillna("")
    df_index_tmp['dupl_list_kor{}'.format(i)] = df_index_tmp['dupl_list_kor{}'.format(i)].fillna("")

In [ ]:
df_index_tmp.iloc[:,30]

In [ ]:
df_index_tmp['stwd_kor{}'.format(1)].fillna("")

In [ ]:

    
    
    
    
#     input_split_kor["num_kor{}".format(i)] = "num_kor{}".format(i)
#     globals()["stwd_kor{}".format(i)] = pd.concat(
#             [
#             df_index.fillna(""), 
#             pd.DataFrame(input_split_kor["stwd_kor{}".format(i)]).fillna(""),
#             pd.DataFrame(input_split_kor["num_kor{}".format(i)]).fillna(""),
#             globals()["dupl_list_kor{}".format(i)].fillna("")
#             ], 

In [ ]:
df_index_tmp = df_index
df_index_tmp 

In [ ]:
for i in range(kor_firts_cnt):
    a = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)].columns = ['index', 'stwd_kor{}'.format(i), 'num_kor{}'.format(i), 'dupl_list_kor{}'.format(i)]
#     tmp['num_kor{}'.format(i)] = a['stwd_kor']
    aa = a.merge(globals()['stwd_kor_uni{}'.format(i)], left_on = 'index', right_on = 'index', how = 'left')
    print('===========================================================================')
    print(a)
    df_index_tmp = df_index_tmp.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        left_on ='index',
        right_on = 'index',
        how = 'left'
    )
df_index_tmp

In [ ]:
df_index_tmp.columns

In [ ]:
tmp = pd.DataFrame()
for i in range(kor_firts_cnt):
#     a = pd.DataFrame()
    a = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
    globals()['stwd_kor_uni{}'.format(i)] = stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor{}'.format(i)]
#     tmp['num_kor{}'.format(i)] = a['stwd_kor']
    tmp = df_index.merge(
        globals()['stwd_kor_uni{}'.format(i)], 
        left_on ='index',
        right_on = 'index',
        how = 'left'
    )
    print(i)
    print('===========================================================================')
    print(tmp)
tmp

In [ ]:
stwd_total_kor_unique[stwd_total_kor_unique['num_kor'] == 'num_kor0']

In [ ]:
stwd_total_kor_unique.loc[stwd_total_kor_unique['num_kor'] == 'num_kor0', 'num_kor']

In [ ]:
unique_stwd_kor['stwd_kor'].drop_duplicates()

In [ ]:
pd.DataFrame(unique_stwd_kor['stwd_kor'].drop_duplicates())

In [ ]:
# 특수기호 검사 : 한, 영
# 소문자 검사 : 한, 영
# 스페이스 제거 : 언제나
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")

# 참고

In [ ]:
############ num_kor(n) / num_eng(n) 기준으로 테이블 분리
############ index 기준으로 left join하여 원상복구

In [ ]:
############## 이거 참고
# db 조건 조회
print(len(stwd_eng7.loc[stwd_eng7['stwd_eng7'] != "", 'stwd_eng7']))
# 특수기호 제거
# for i in range(len(input_test['영문약어명'])):
#     tmp = input_test['영문약어명'][i].translate(str.maketrans('', '', string.punctuation))
#     print(tmp)
print(string.punctuation)

In [ ]:
# # 특수문자 제거
# for i in range(len(input_test['영문약어명'])):
#     tmp = input_test['영문약어명'][i].translate(str.maketrans('', '', string.punctuation))
#     i += i
#     print(tmp)
# i

In [ ]:
# # -*- coding: utf-8 -*-
# import sys
# import re

# def isHangul(text):
#     #Check the Python Version
#     pyVer3 =  sys.version_info >= (3, 0)

#     if pyVer3 : # for Ver 3 or later
#         encText = text
#     else: # for Ver 2.x
#         if type(text) is not unicode:
#             encText = text.decode('utf-8')
#         else:
#             encText = text

#     hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
#     return hanCount > 0

# 한글 영문 추리기1
# if __name__ == '__main__':
#     han = '한글'
#     han2 = u'한글'
#     input = 'English1234'  #utf-8
#     input2 = u'English1234' #unicode
#     #(major_ver, minor_ver) = (sys.version_info).split(',')
#     print(han + " is " + str(isHangul(han)))
#     print(han2 + " is " + str(isHangul(han2)))
#     print(input + " is " + str(isHangul(input)))
#     print(input2 + " is " + str(isHangul(input2)))

# 한글 영문 추리기2
# i = 23
# print(stwd_total['stwd_eng'][i])
# print(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', stwd_total['stwd_eng'][i]))
# print(re.findall('[A-Za-z]', stwd_total['stwd_eng'][i]))

In [ ]:
# left join
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )

# 카운트 일치 여부 점검
index_cnt = len(df_index['index'])
if index_cnt != len(stwd_total0):
    print("index 와 joined_date count불일치")
else:
    print("count 일치")
stwd_total5

In [ ]:
# merge
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.concat(
        [
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)]
        ],
        axis = 1        
    )
    print(i)
# print(name_cnt)
# print(name_cnt_diff_list)


stwd_total1

In [ ]:
for i in list(set(range(name_cnt)) - set(range(name_cnt_diff))):
    print(i)


In [ ]:
list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))

In [ ]:
input_split_eng

In [ ]:
input_split_eng['stwd_eng6'] = None
input_split_eng['stwd_eng6']

In [ ]:
input_split_eng['stwd_eng6']

In [ ]:
# 한글/영문 컬럼 수 비교하기
name_cnt_kor = len(name_list_kor)
name_cnt_eng = len(name_list_eng)

if name_cnt_kor == name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = None
elif name_cnt_kor > name_cnt_eng:
    name_cnt = name_cnt_eng
    name_cnt_diff_list = list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))
    for i in name_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)]),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)])
            ],
            axis = 1
        )
        
        
        
elif name_cnt_kor < name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = list(set(range(name_cnt_eng)) - set(range(name_cnt_kor)))
    for i in name_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)]),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)])
            ],
            axis = 1
        )
        

stwd_eng7


In [ ]:
stwd_eng6

In [ ]:
# 한글/영문 컬럼 수 비교하기
name_cnt_kor = len(name_list)
name_cnt_eng = len(name_list_eng)

if name_cnt_kor == name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = None
elif name_cnt_kor > name_cnt_eng:
    name_cnt = name_cnt_eng
    name_cnt_diff_list = list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))
    for i in name_cnt_diff_list:
        globals()['stwd_eng{}'.format(i)] = None
elif name_cnt_kor < name_cnt_eng:
    name_cnt = name_cnt
    name_cnt_diff_list = list(set(range(name_cnt_eng)) - set(range(name_cnt_kor)))
    
    

    
    
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.concat(
        [
        globals()['stwd{}'.format(i)],
        globals()['stwd_eng{}'.format(i)]
        ],
        axis = 1        
    )
    print(i)
# print(name_cnt)
# print(name_cnt_diff_list)

In [ ]:
# 리스트 - 리스트
list1 = [1,2]
list2 = [1,2,3]
list(set(list2) - set(list1))
list(set(list1) - set(list2))
list(set(list1).difference(set(list2)))
list(set(range(10)) - set(range(2)))

In [ ]:
list(set(range(10)) - set(range(2)))

In [ ]:
list(set(range(2)) - set(range(10)))

In [ ]:
# 용어명 분리
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
name_list = []
for i in range(len(input_split.columns)):
    name_list.append("stwd{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = name_list)
# 인덱스 컬럼 추가
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']
input_split= pd.concat([input_split, df_index], axis = 1)
print(input_split.shape)
input_split.head(5)

In [ ]:
# 영문명 분리
input_test['영문약어명'] = input_test['영문약어명'].str.replace(" ", "")
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist())
name_list_eng = []
for i in range(len(input_split_eng.columns)):
    name_list_eng.append("stwd_eng{}".format(i))
print(name_list_eng)
# 컬럼명 만들기
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist(), columns = name_list_eng)
# 인덱스 컬럼 추가
# df_index = pd.DataFrame(input_test['No'])
# df_index.columns = ['index']
# input_split_eng= pd.concat([input_split_eng, df_index], axis = 1)
print(input_split_eng.shape)
input_split_eng.head(5)

In [ ]:
name_list = []
for i in range(len(input_split_eng.columns)):
    name_list_eng.append("stwd_eng{}".format(i))

In [ ]:
if len(name_list) == len(name_list_eng):
    name_list_cnt = len(name_list)
elif len(name_list) > len(name_list_eng):
    name_list_cnt = len(name_list)
elif len(name_list) < len(name_list_eng):
    name_list_cnt = len(name_list_eng)
    
for i in range(name_list_cnt):
    print(i)

In [ ]:
if len(name_list) == len(name_list_eng):
    print(1)
elif len(name_list) > len(name_list_eng):
    print(2)    
elif len(name_list) < len(name_list_eng):
    print(3)

In [ ]:
len(name_list_eng)

In [ ]:
len(name_list)

In [ ]:
# 용어명 분리
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
name_list = []
for i in range(len(input_split.columns)):
    name_list.append("stwd{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = name_list)
print(input_split.shape)
input_split.head(5)

In [ ]:
# 단어별 개수 세기(네이밍 컬럼)
dupl_list = []
dupl_cnt = []
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []
    for j in range(len(input_split["stwd{}".format(i)])):
        dupl_cnt = input_split["stwd{}".format(i)].to_list().count(input_split["stwd{}".format(i)][j])
        globals()["dupl_list{}".format(i)].append(dupl_cnt)
# df_dupl 힌즐 세우기
df_dupl = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)]
    df_dupl = pd.concat([df_dupl, pd.DataFrame(globals()["dupl_list{}".format(i)])], axis = 0)
df_dupl.columns = ['dupl']
df_dupl

In [ ]:
# df_index 한줄 세우기
# df_index = pd.DataFrame(input_test['No'])
df_index = pd.DataFrame()
for i in range(len(input_split.columns)):
#     globals()["df_index{}".format(i)] = pd.DataFrame(input_test['No'])
    df_index = pd.concat([df_index, pd.DataFrame(input_test['No'])])
df_index.columns = ['index']
df_index

In [ ]:
# df_stwd 힌즐 세우기
df_stwd = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["df_stwd{}".format(i)] = input_split["stwd{}".format(i)]
    df_stwd = pd.concat([df_stwd, globals()["df_stwd{}".format(i)]])
df_stwd.columns = ['stwd']
df_stwd

In [ ]:
# df_chk
df_chk = pd.concat([df_stwd, df_index], axis=1).reset_index() # 인덱스 초기화
df_chk = df_chk[['stwd', 'index']]
print(df_chk.shape)
df_chk.tail(5)

In [ ]:
### 한글 단어 분리

In [ ]:
# 영문명 분리
input_test['영문약어명'] = input_test['영문약어명'].str.replace(" ", "")
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist())
name_list = []
for i in range(len(input_split_eng.columns)):
    name_list.append("stwd_eng{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist(), columns = name_list)
print(input_split_eng.shape)
input_split_eng.head(5)

In [ ]:
# df_index 한줄 세우기
df_index_eng = pd.DataFrame()
for i in range(len(input_split_eng.columns)):
    df_index_eng = pd.concat([df_index_eng, pd.DataFrame(input_test['No'])])
df_index_eng.columns = ['index']
df_index_eng

In [ ]:
# df_stwd_eng 힌즐 세우기
df_stwd_eng = pd.DataFrame()
for i in range(len(input_split_eng.columns)):
    globals()["df_stwd_eng{}".format(i)] = input_split_eng["stwd_eng{}".format(i)]
    df_stwd_eng = pd.concat([df_stwd_eng, globals()["df_stwd_eng{}".format(i)]])
df_stwd_eng.columns = ['stwd_eng']
df_stwd_eng

In [ ]:
# df_chk_eng
df_chk_eng = pd.concat([df_stwd_eng, df_index_eng], axis=1).reset_index() # 인덱스 초기화
df_chk_eng = df_chk_eng[['stwd_eng', 'index']]
print(df_chk_eng.shape)
df_chk_eng.tail(5)

In [ ]:
### 영문 단어 분리

In [ ]:
df_chk.merge(df_chk_eng, on='index' 

In [ ]:
# df_chk_uni
df_chk_uni = df_chk[df_chk['stwd'].duplicated()==False].reset_index()
df_chk_uni = df_chk_uni[['stwd', 'index']]
df_chk_uni.tail(5)

In [ ]:
# unique 단어 개수 세기
dupl_list = []
dupl_cnt = []
for i in range(len(df_chk_uni["stwd"])):
    dupl_cnt = df_chk_uni["stwd"].to_list().count(df_chk_uni["stwd"][i])
    dupl_list.append(dupl_cnt)

len(dupl_list)

In [ ]:
# 횟수 병합
df_chk_uni = pd.concat([df_chk_uni, pd.DataFrame(dupl_list)], axis = 1)
df_chk_uni.columns = ['stwd', 'index', 'dupl_cnt']
print(df_chk_uni[df_chk_uni['dupl_cnt'] != 1])
print(df_chk_uni[df_chk_uni['dupl_cnt'] == None])
df_chk_uni.tail(5)

In [ ]:
# 점검 , 변환
# ab['dupl_cnt'].info()
# ab['dupl_cnt'].astype(int)

In [ ]:
# df_chk_uni
df_chk_dupl = df_chk[df_chk['stwd'].duplicated()].reset_index()
df_chk_dupl = df_chk_dupl[['stwd', 'index']]
df_chk_dupl.tail(5)

In [ ]:
# dupl 단어 개수 세기
dupl_list = []
dupl_cnt = []
for i in range(len(df_chk_dupl["stwd"])):
    dupl_cnt = df_chk_dupl["stwd"].to_list().count(df_chk_dupl["stwd"][i])
    dupl_list.append(dupl_cnt)

len(dupl_list)

In [ ]:
# 횟수 병합
df_chk_dupl = pd.concat([df_chk_dupl, pd.DataFrame(dupl_list)], axis = 1)
df_chk_dupl.columns = ['stwd', 'index', 'dupl_cnt']
print(df_chk_dupl[df_chk_dupl['dupl_cnt'] == 1])
print(df_chk_dupl[df_chk_dupl['dupl_cnt'] == None])
df_chk_dupl.tail(5)

In [ ]:
df_chk_dupl[df_chk_dupl['stwd'] == '매수']

In [ ]:
cc = df_chk_dupl.merge(input_test, left_on = 'index', right_on = 'No', how='left')
cc = cc[cc['stwd'].duplicated()==False]
print(cc.shape)
cc.head(3)

In [ ]:
df_chk_dupl.to_excel(input_path+"dsfadas.xlsx")

In [ ]:
cc = df_chk_dupl['stwd'].drop_duplicates()
(cc.shape)
cc

In [ ]:
cc.to_excel(input_path+"cccccccccccc.xlsx")

In [ ]:
df_chk_dupl['stwd'].drop_duplicates()

In [ ]:
ccc = df_chk_dupl.merge(input_test, left_on = 'index', right_on = 'No', how='left')
ccc = ccc[ccc['stwd'].duplicated()]
print(ccc.shape)
ccc.head(3)

In [ ]:
ccc.to_excel(input_path+"cccccccccc3cc.xlsx")

In [ ]:
a[a['stwd'].duplicated()]

In [ ]:
a[pd.DataFrame(a['stwd']).duplicated(keep='first')]

In [ ]:
a['stwd'].drop_duplicates()

In [ ]:
a = pd.concat([df_stwd, df_dupl, df_index], axis=1)
a

In [ ]:
tOK = a.loc[a['dupl'] == 1]
# tOK['stwd'].dropna()
tOK[tOK['stwd'] == '검토']
tOK
tNO = a.loc[a['dupl'] > 1]
# tNO['stwd'].dropna()
tNO[tNO['stwd'] == '검토']
tNO

In [ ]:
tOK['stwd'].drop_duplicates()

In [ ]:
print(368 + 412)
tNO['stwd'].drop_duplicates()

In [ ]:
a['stwd'].drop_duplicates()

In [ ]:
t2['stwd'].drop_duplicates()

In [ ]:
a.to_excel(input_path+"\\dfdf.xlsx")

In [ ]:
tOK.to_excel(input_path+"\\dfdfOK.xlsx")

In [ ]:
tNO.to_excel(input_path+"\\dfdfNO.xlsx")

In [ ]:
tNO['stwd'].drop_duplicates().to_excel(input_path+"\\dfdfNO_DUP.xlsx")

In [ ]:
t2['stwd'].drop_duplicates().tail(30)

In [ ]:
# t2 = a[a['stwd'].duplicated(keep='first')]
t2 = a[a['stwd'].duplicated(False)]
t2[t2['stwd'] == '검토']
# t2['stwd'].dropna()
t2

In [ ]:
t2.drop_

In [ ]:
a[a['stwd'] == '검토']

In [ ]:
a.head(20)

In [ ]:
a['stwd'].duplicated(keep='first').head(20)
# t#['stwd'].dropna()
# t3[t3['stwd'] == '검토']

In [ ]:
# # concat 을 위한 DataFame 묶기 (o 시리즈여야 한곳에 정렬됨! 데이터 프레임 안됨)
# for i in range(len(input_split.columns)):
#     globals()["df_stwd{}".format(i)] = ["duple_list{}".format(i)]
# df_stwd0

In [ ]:
# concat 을 위한 DataFame 묶기 (o 시리즈여야 한곳에 정렬됨! 데이터 프레임 안됨)
for i in range(len(input_split.columns)):
    globals()["df_stwd{}".format(i)] = input_split["stwd{}".format(i)]
df_stwd0

In [ ]:
# df_index 묶기
for i in range(len(input_split.columns)):
    globals()["df_index{}".format(i)] = input_split["stwd{}".format(i)]
df_stwd0

In [ ]:
df_stwd = pd.concat([df_stwd, df_index], axis=1)
df_stwd

In [ ]:
pd.concat([input_split, pd.DataFrame(input_test['No'])], axis=1)


In [ ]:
# df_stwd.tail(500)

In [ ]:
list = []
for i in range(len(input_split.columns)):
    df = pd.concat([globals()["df_stwd{}".format(i)]])
df

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    df = pd.concat([df, globals()["df_stwd_df{}".format(i)]])
df 

In [ ]:
df

In [ ]:
df_dupl_cnt = pd.DataFrame()
for in in range(len(input_split.coulmns):
    df_dupl_cnt = pd.concat([df, globals()["df{}".format(i)]])

In [ ]:
dupl_list0

In [ ]:
# 컬럼 재정렬
list = []
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]
    list.append("stwd{}".format(i))
    list.append("dupl_cnt{}".format(i))
input_split = input_split[list]
input_split

In [ ]:
df_stwd = pd.DataFrame()
for i in range(len(input_split.columns)):
    df_stwd = pd.concat([df, globals()["df{}".format(i)]])
df_stwd 

In [ ]:
pd.DataFrame

In [ ]:
type(input_split["stwd{}".format(0)])

In [ ]:
a = pd.Series
a

In [ ]:
a

In [ ]:
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = input_split["stwd{}".format(i)]
    list.append('df{}'.format(i))
list

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = pd.DataFrame(input_split["stwd{}".format(i)])
#     list.append('df{}'.format(i))
# list

In [ ]:
list = []
for i in range(len(input_split.columns)):
    list.append("df{}".format(i))
list

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    df = pd.concat([df, globals()["df{}".format(i)]])
df

In [ ]:
concat([df0, df0,df0,df0,df0,df0,df0,df0,])

In [ ]:
list = ",".join(list)
list
list[5]

In [ ]:
",".join(list[0])

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    ccc = pd.concat(df, ",".join(list[i]))
df = pd.DataFrame()
df = pd.concat([df, df0])

In [ ]:
df = pd.DataFrame()
df = pd.concat([df, df0])

In [ ]:
",".join(list)

In [ ]:
list = ",".join(list)
list

In [ ]:
list

In [ ]:
df0

In [ ]:
list = [df0, df1, df2, df3, df4, df5]
pd.concat(list)

In [ ]:
pd.concat([df0, df1, df2, df3, df4, df5])


In [ ]:
input_split

In [ ]:
stwd_list

In [ ]:
stwd_list = input_split.columns.to_list()
for i in range(len(input_split.columns)):
    duplicated = pd.concat(stwd_list, ignore_index=True)

In [ ]:
stwd_list = input_split.columns.to_list()
pd.concat(pd.DataFrame(input_split[stwd_list]), ignore_index = True)

In [ ]:
list = []
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = input_split["stwd{}".format(i)]
    list.append(globals()["df{}".format(i)])

In [ ]:
pd.DataFrame(list)

In [ ]:
pd.DataFrame(input_split["stwd1"])

In [ ]:
input_split["stwd0"]

In [ ]:
pd.concat(pd.DataFrame(input_split["stwd0"]), pd.DataFrame(input_split["stwd1"]))

In [ ]:
df0
list

In [ ]:
globals("df0")

In [ ]:
pd.concat([df0, df1])

In [ ]:
pd.concat([df0, df1])

In [ ]:
for i in range(len(input_split.columns)):
#     pd.concat(globllist[i])
    print(list[i])

In [ ]:
for i in range(len(input_split.columns)):
#     pd.concat(["df{}".format(i)])
#     pd.concat("df{}".format(i))
    pd.concat(list)  
#     print("df{}".format(i))

In [ ]:
for i in range(len(input_split.columns)):
    print("df{}".format(i))

In [ ]:
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = pd.DataFrame(globals()["df{}".format(i)])

In [ ]:
list = []
for i in range(len(input_split.columns)):
    list.append("df{}".format(i))

In [ ]:
df1

In [ ]:
pd.concat(list, axis=1 )
#           ignore_index=True)

In [ ]:
for i in range(len(input_split.columns)):
    pd.concat(globals()["df{}".format(i)])

In [ ]:
pd.concat([df0, df1])

In [ ]:
stwd_list = input_split.columns.to_list()
for i in range(len(input_split.columns)):
    pd.concat(pd.DataFrame(input_split["stwd{}".format(i)]))
list

In [ ]:
input_split["stwd0"]

In [ ]:
pd.DataFrame(input_split["stwd{}".format(0)])

In [ ]:
cc = pd.DataFrame(input_split["stwd{}".format(i)])

In [ ]:
aa = pd.DataFrame
for i in range(len(input_split.columns)):
    aa = aa.append(cc)
aa

In [ ]:
for i in range(len(input_split.columns)):
    a = pd.concat([input_split[list[i]]])
a

In [ ]:
a

In [ ]:
pd.concat(list)

In [ ]:
for i in range(len(input_split.columns)):
    pd.concat([globals["input_split{}".format(i)]])

In [ ]:
# 컬럼 재정렬
list = []
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]
    list.append("stwd{}".format(i))
    list.append("dupl_cnt{}".format(i))
input_split = input_split[list]
input_split

In [ ]:
input_split.columns

In [ ]:
for i in range(int(len(input_split.columns)/2)):
    aaa= pd.concat(["stwd{}".format(i)])
aaa

In [ ]:
input_split[0]

In [ ]:
input_split[0]

In [ ]:
input_split[0].count("가입")

In [ ]:
dupl_list =[]
dupl_list

In [ ]:
len(input_split.columns)

In [ ]:
input_split[0].to_list().count(input_split[0][11])

In [ ]:
globals()["stwd{}".format(i)]

In [ ]:
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []

In [ ]:
input_split[0].to_list().count(input_split[0][3])

In [ ]:
# 성공
dupl_list = []
dupl_cnt = []


for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []
    for j in range(len(input_split[i])):
        dupl_cnt = input_split[i].to_list().count(input_split[i][j])
        globals()["dupl_list{}".format(i)].append(dupl_cnt)
#         print(dupl_cnt)
#         print("dupl_list{}".format(i))




In [ ]:
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]

In [ ]:
input_split["dupl_cnt0"]

In [ ]:
# input_split
input_split[input_split["dupl_cnt0"] >= 2][["dupl_cnt0","dupl_cnt1",0]]

In [ ]:
input_split.loc[:,"dupl_cnt1"] = dupl_list1

In [ ]:
input_split

In [ ]:
for i in range(len(input_split.columns)):
    print(globals()["dupl_list{}".format(i)])

In [ ]:
a= input_split.assign(dd =  dupl_list3)
# df.insert(3,"city",['Lahore','Dehli','New York'],True)
a

In [ ]:
input_split.insert(8,"dd",dupl_list3,True)
input_split

In [ ]:
input_split[2].to_list().count('역')

In [ ]:
input_split[0].to_list()

In [ ]:
dupl_list3

In [ ]:
input_split.insert(8,"dd",dupl_list3,True)

In [ ]:
input_split

In [ ]:
pd.DataFrame(input_split[3].to_list(), dupl_list3)

In [ ]:
print(len(dupl_list))
dupl_list

In [ ]:
# type(input_split[0])
input_split[0].to_list().count('가입')

In [ ]:
# 성공
b = []
for i in range(len(input_split[0])):
    a = 0
    for j in input_split[0]:
        b.append(a)
        if input_split[0][i] == j:
            a += 1
            print(input_split[0][i] +"&&"+j+"_중복")
           
            print(a)


In [ ]:
len(b)

In [ ]:
# 성공
a = 0
for i in range(len(input_split[0])):
    for j in input_split[0]:
        if input_split[0][i] == j:
            print(input_split[0][i] +"&&"+j+"_중복")
            a += 1
            print(a)


In [ ]:
# 성공
a =0
for i in input_split[0]:
    for j in input_split[0].unique():#중복 
        if i == j:
            print(i +"&&"+j+"_중복")
            a += 1
            print(a)
#         else:
#             print(i+j)
    
    
    
#     if i == '가입':
#         print(i+"_성공")
#     else:
#         print(i)

In [ ]:
for i in input_split[0]:
    if i == '가입':
        print(i+"_성공")
    else:
        print(i)

In [ ]:
input_split[0].str == "가입"

In [ ]:
for i in a
    input_split[0] == i
    

In [ ]:
input_split[0]

In [ ]:
a = input_split[0].unique()
a

In [ ]:
len(input_split.columns)

In [ ]:
input_processing = pd.concat([input_split, input_test])
print(input_processing.columns)
input_processing.head(5)

In [ ]:
input_process["No"]

In [ ]:
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : row[10])
# # stwd1

In [ ]:
aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = ['a' ,'b', 'c', 'd','a' ,'b', 'c', 'd'])
aa

In [ ]:
aaa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
aaa

In [ ]:
aaa[1]

In [ ]:
aaa.columns

In [ ]:
aa['a']

In [ ]:
aa.columns

In [ ]:
for i in range(len()):
    aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = globals()["stwd{}".format(i)])
    aa

In [ ]:

try:    
    for i in range(10):
         globals()["stwd{}".format(i)] = pd.DataFrame({"stwd{}".format(i) : []})
except:
    pass
# print(stwd0)

    
        
    
# pd.DataFrame({"stwd0" : []})
        

In [ ]:
print(input_test['용어명 분리'].str.split('_', maxsplit=1))

In [ ]:
# try:
for i in range(10):
    globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_').map(lambda row : row[i])

input_test['용어명 분리'].str.split('_').map(lambda row : row[0])

In [ ]:
pd.DataFrame(input_test['용어명 분리'].str.split('_'))


In [ ]:
input_test['용어명 분리'].str.split('_').tolist() 

In [ ]:
aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = ['a' ,'b', 'c', 'd','a' ,'b', 'c', 'd'])
aa

In [ ]:
input_test['용어명 분리'].str.split('_').map(lambda x:x[-1])


In [ ]:
a= pd.DataFrame(input_test['용어명 분리'].str.split('_'))
a

In [ ]:
a.str.get(0)

In [ ]:
input_test['용어명 분리'].str.split('_')[0][0]


In [ ]:
input_test['용어명 분리'].str.split('_').map(lambda row : row[-1])

In [ ]:
stwd

In [ ]:
stwd2

In [ ]:
# i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
#     globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_', maxsplit = -1).map(lambda row : row[i])

# stwd0
# stwd1

# stwd0 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[0]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd2 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd3 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd4 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd5 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd6 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd7 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd8 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd9 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
try:
    stwd10 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[10]))
except:
    pass


In [ ]:
# i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
num = 10
for i in range(10):
#     if input_test['용어명 분리'].str.split("_").map(lambda row : row[i]) 
    
    globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_', maxsplit = -1).map(lambda row : row[i])

stwd0
# stwd1


# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))


In [ ]:
stwd1

In [ ]:
input_f = os.path.join(os.getcwd(), "input")
print(input_f)
# pd.read_excel(input_f+"test.xlsx", sheet_name="input_test", header=0)
sheet_name = "input_test"
pd.read_excel("C:/test.xlsx", sheet_name=sheet_name, header=0)

# current_path = os.path.dirname(input_f)
# input_f = os.path.join(current_path, "input")
print()
# current_path = os.path.dirname()
# print(current_path)
print(os.getcwd())

In [ ]:
print(os.getcwd())
# print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))